In [4]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import pandas as pd
from datasets import Dataset
import json

In [5]:
transformers.logging.set_verbosity_info()

In [29]:
# # Load the .jsonl file
# with open("/content/drive/MyDrive/Data/subset_cnn_dailymail.jsonl", "r") as f:
#     ds_list = [json.loads(line) for line in f]

# print(f"Loaded {len(ds_list)} rows.")

Loaded 100000 rows.


In [2]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpo

In [30]:
# def preprocess_function(examples):
#     # Use 'article' as input text
#     inputs = examples['article']
#     # Tokenize the inputs (article content)
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True)

#     # Use 'highlights' as the target for summarization
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples['highlights'], max_length=128, truncation=True)

#     # Add the labels to model inputs
#     model_inputs['labels'] = labels['input_ids']
#     return model_inputs

In [31]:
# tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [32]:
# Save the tokenized dataset to disk
# tokenized_dataset.save_to_disk("/content/drive/MyDrive/Data/tokenized_subset_cnn_dailymail")

Saving the dataset (0/2 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [3]:
from datasets import load_from_disk

# Load the dataset from disk
tokenized_dataset = load_from_disk("/content/drive/MyDrive/Data/tokenized_subset_cnn_dailymail")

In [4]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

train_dataset = split_dataset['train']
val_dataset = split_dataset['test']
tokenized_train_dataset = train_dataset.remove_columns(['article', 'highlights', 'id'])
tokenized_val_dataset = val_dataset.remove_columns(['article', 'highlights', 'id'])

print(f"Train size: {len(tokenized_train_dataset)}")
print(tokenized_train_dataset.column_names)
print(f"Validation size: {len(tokenized_val_dataset)}")

Train size: 90000
['input_ids', 'attention_mask', 'labels']
Validation size: 10000


In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Data/results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    remove_unused_columns=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator = data_collator
)

In [ ]:
trainer.train()